# 멜론에서 차트 가져오기

In [14]:
from bs4 import BeautifulSoup as bs              # 데이터파싱 라이브러리
import urllib
import urllib.request as req                     # 데이터수신 라이브러리
import pandas as pd                              # 데이터정리 라이브러리 
import datetime                                  # 날짜데이터 라이브러리
import re                                        # 정규표현식 라이브러리

In [15]:
melon_list = pd.DataFrame(columns = ['artist',
                                     'artist_code',
                                     'title',
                                     'title_code',
                                     'rank',
                                     'date',
                                     'hour',
                                     'site'])

melon_list.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 8 columns):
artist         0 non-null object
artist_code    0 non-null object
title          0 non-null object
title_code     0 non-null object
rank           0 non-null object
date           0 non-null object
hour           0 non-null object
site           0 non-null object
dtypes: object(8)
memory usage: 0.0+ bytes


In [16]:
dt_now = datetime.datetime.now() 
print(dt_now)

2019-01-15 14:02:10.728055


아래와 같이 헤더를 만들어 보내면 서버는 요청된 것을 윈도우- 크롬에서 요청한 것으로 인지한다.

In [17]:
url = "http://www.melon.com/chart/index.htm"
header_ = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36"

request = req.Request(url, headers = {'User-Agent':header_})
html = req.urlopen(request)
page = bs(html.read(), 'lxml')

print(html.getcode()) # 200이 뜨면 잘 불러온 것!

200


In [18]:
page.find("div", {'class':'ellipsis rank01'})

<div class="ellipsis rank01"><span>
<a href="javascript:melon.play.playSong('19030101',31532643);" title="넘쳐흘러 재생">넘쳐흘러</a>
</span></div>

In [19]:
song_title = page.find("div", {'class':'ellipsis rank01'}).get_text().strip()
song_artist = page.find("span", {'class':'checkEllipsis'}).get_text().strip()

print(song_title)
print(song_artist)

page.find("div", {'class':'ellipsis rank01'})

넘쳐흘러
엠씨더맥스


<div class="ellipsis rank01"><span>
<a href="javascript:melon.play.playSong('19030101',31532643);" title="넘쳐흘러 재생">넘쳐흘러</a>
</span></div>

In [20]:
songlists50 = page.find_all('tr', {'class':'lst50'})
songlists100 = page.find_all('tr', {'class':'lst100'})

for half in [songlists50, songlists100]:
    for songs in half:
        song_artist = songs.find("span", {'class':'checkEllipsis'}).get_text().strip()

        ac = songs.find('span', {'class':'checkEllipsis'})
        p = re.compile('[0-9]+')
        song_artist_code = int(p.findall(str(ac))[0])

        song_title = songs.find("div", {'class':'ellipsis rank01'}).get_text().strip()
        song_title_code = int(songs['data-song-no'])
        song_rank = int(songs.find("span", {'class' : "rank "}).get_text())
        song_date = dt_now.strftime('%Y-%m-%d')
        song_hour = dt_now.strftime('%H')
        song_site = "melon"

    
        melon_list = melon_list.append(
            {'artist':song_artist,
             'artist_code':song_artist_code,
             'title' : song_title,
             'title_code' : song_title_code,
             'rank' : song_rank,
             'date' : song_date,
             'hour' : song_hour,
             'site' : song_site}, ignore_index = True)  

In [21]:
melon_list.to_csv(dt_now.strftime("%Y_%m_%d_%H")+".csv", index=False, encoding = 'euc-kr')

#### 앨범사진 src 가져오기

In [22]:
a= []
for name in page.find("a", {'class':'image_typeAll'}).children:
    a.append(name)
a[1]['src']

'https://cdnimg.melon.co.kr/cm/album/images/102/38/710/10238710_500.jpg/melon/resize/120/quality/80/optimize'

In [23]:
urllib.request.urlretrieve(a[1]['src'], "{0}.jpg".format(song_title))

('묻고싶다 (One Love).jpg', <http.client.HTTPMessage at 0x29ddb1b3358>)